# Set up the environment

In [1]:
%load_ext autoreload
%autoreload 2

# Monkey with the path, so that we can import like rational humans
try:
    import deepseg as ds
except:
    import sys, os
    sys.path.insert(0, os.path.abspath('./../../trace/'))
    
dataset_folder = './../../../datasets/'

In [2]:
import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [3]:
import tensorflow as tf

In [4]:
import trace.dataset as dset
import trace.alignment as al
import trace.sampling as samp
import trace.train as train

In [5]:
isbi_folder = dataset_folder  + 'isbi/'
isbi_dset = dset.ISBIDataset(isbi_folder)

In [6]:
def create_validation_set(sess, sampler, validation_images):
    assert(len(validation_images.shape) == 4)
    
    val_set = []
    
    # Every pair of images
    for i in range(len(validation_images) - 1):
        input_pair = validation_images[i:i+2, :, :, 0:1]
        ref_op, sec_op, corr_op, transf_op = sampler.get_sample_funcs()
        ref, sec, corr, transf = sess.run([ref_op, sec_op, corr_op, transf_op],
                                          feed_dict={sampler.aligned_sample: input_pair})
        val_set.append(samp.LocalizationExample(ref, sec, corr, transf))
        
    return val_set

# Translation, train on transformations, fixed dset

Define the translation parameters.

In [11]:
# Reset the graph
tf.reset_default_graph()

In [12]:
translation = True
max_shift = 10
rotation = False
max_angle = None

full_isbi_dim = 512
isbi_sampler = samp.SimpleEMInputSampler(dataset=isbi_dset, sample_shape=[1, 2, full_isbi_dim, full_isbi_dim, 1], batch_size=1)

sampler = samp.LocalizationSampler(dset_sampler=isbi_sampler, rotation_aug=rotation,
                                   max_angle=max_angle, translation_aug=True, max_shift=max_shift)

val_images = sampler.dset_sampler.get_validation_set()

trans_transformer = al.ConvTranslationSpatialTransformer(in_dim=full_isbi_dim, l1_n=24, l2_n=36, l3_n=48, 
                                                         l4_n=64, fc1_n=512, max_shift=max_shift, trainable=True)

In [13]:
with tf.Session() as sess:
    sampler.initialize_session_variables(sess)
    
    # Define a validation set
    v_set = create_validation_set(sess, sampler, val_images[0])
    
    # Create a trainer
    trainer = train.LocalizationTrainer(trans_transformer, 
                                        isbi_folder + '/realignment/train_transform/translation/run-20/')
    
    trainer.train_on_transformations(sess, 50000, sampler, lr=0.00001, validation_set=v_set)
    


Instructions for updating:
Use `tf.global_variables_initializer` instead.
0
theta_pred: 1.000000, 0.000000, -13.247581, 0.000000, 1.000000, 1.122195, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 13.000000, 0.000000, 1.000000, 5.000000, 0.000000, 0.000000
Computed images
Validation pixel error 0.195938
Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-1/model.ckpt
10
20
30
40
50
60
70
80
90
100
theta_pred: 1.000000, 0.000000, 1.842049, 0.000000, 1.000000, -1.768964, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 5.000000, 0.000000, 1.000000, 11.000000, 0.000000, 0.000000
Computed images
110
120
130
140
150
160
170
180
190
200
theta_pred: 1.000000, 0.000000, -2.241520, 0.000000, 1.000000, -1.286523, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 18.000000, 0.000000, 1.000000, 0.000000, 0.000000, 0.000000
Computed images
210
220
230
240
250
260
270
280
290
300
theta_pred: 1.000000, 0.000000, 0.116543, 0.000000, 1.000000,

KeyboardInterrupt: 

# Translation, train on translation, augmented dataset

In [15]:
# Reset the graph
tf.reset_default_graph()

In [ ]:
translation = True
max_shift = 10
rotation = False
max_angle = None

reduced_isbi_dim = 511
isbi_rand_sampler = samp.EMDatasetSampler(dataset=isbi_dset, sample_shape=[2, reduced_isbi_dim, reduced_isbi_dim, 1], batch_size=1)

aug_sampler = samp.LocalizationSampler(dset_sampler=isbi_rand_sampler, rotation_aug=rotation,
                                   max_angle=max_angle, translation_aug=True, max_shift=max_shift)

val_images, _, _ = aug_sampler.dset_sampler.get_validation_set()

trans_transformer = al.ConvTranslationSpatialTransformer(in_dim=reduced_isbi_dim, l1_n=24, l2_n=36, l3_n=48, 
                                                         l4_n=64, fc1_n=512, max_shift=max_shift, trainable=True)

In [ ]:
with tf.Session() as sess:
    aug_sampler.initialize_session_variables(sess)
    
    # Define a validation set
    v_set = create_validation_set(sess, aug_sampler, val_images[0])
    
    # Create a trainer
    trainer = train.LocalizationTrainer(trans_transformer, 
                                        isbi_folder + '/realignment/train_transform/translation/run-21/')
    
    trainer.train_on_transformations(sess, 50000, aug_sampler, lr=0.00001, validation_set=v_set)
    


Instructions for updating:
Use `tf.global_variables_initializer` instead.
0
theta_pred: 1.000000, 0.000000, 6.058322, 0.000000, 1.000000, 1.355079, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 10.000000, 0.000000, 1.000000, 9.000000, 0.000000, 0.000000
Computed images
Validation pixel error 0.196308
Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
10
20
30
40
50
60
70
80
90
100
theta_pred: 1.000000, 0.000000, 1.873494, 0.000000, 1.000000, -0.943186, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 12.000000, 0.000000, 1.000000, 12.000000, 0.000000, 0.000000
Computed images
110
120
130
140
150
160
170
180
190
200
theta_pred: 1.000000, 0.000000, -0.453917, 0.000000, 1.000000, 0.643571, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -14.000000, 0.000000, 1.000000, 5.000000, 0.000000, 0.000000
Computed images
210
220
230
240
250
260
270
280
290
300
theta_pred: 1.000000, 0.000000, -0.836181, 0.000000, 1.000000

2950
2960
2970
2980
2990
3000
theta_pred: 1.000000, 0.000000, 0.146741, 0.000000, 1.000000, -0.160386, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 2.000000, 0.000000, 1.000000, -1.000000, 0.000000, 0.000000
Computed images
Validation pixel error 0.192424
Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
theta_pred: 1.000000, 0.000000, -0.142915, 0.000000, 1.000000, -0.162319, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -9.000000, 0.000000, 1.000000, -10.000000, 0.000000, 0.000000
Computed images
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
theta_pred: 1.000000, 0.000000, -0.066122, 0.000000, 1.000000, -0.118911, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -7.000000, 0.000000, 1.000000, -9.000000, 0.000000, 0.000000
Computed images
3210
3220
3230
3240
3250
3260
3270
3280
3290
3300
theta_pred: 1.000000, 0.000000, -0.000529, 0.000000, 1.000000, -

Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
6010
6020
6030
6040
6050
6060
6070
6080
6090
6100
theta_pred: 1.000000, 0.000000, -0.151290, 0.000000, 1.000000, -0.022858, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -1.000000, 0.000000, 1.000000, -8.000000, 0.000000, 0.000000
Computed images
6110
6120
6130
6140
6150
6160
6170
6180
6190
6200
theta_pred: 1.000000, 0.000000, -0.151233, 0.000000, 1.000000, -0.026689, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 1.000000, 0.000000, 1.000000, 13.000000, 0.000000, 0.000000
Computed images
6210
6220
6230
6240
6250
6260
6270
6280
6290
6300
theta_pred: 1.000000, 0.000000, -0.219395, 0.000000, 1.000000, 0.030435, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 4.000000, 0.000000, 1.000000, -5.000000, 0.000000, 0.000000
Computed images
6310
6320
6330
6340
6350
6360
6370
6380
6390
6400
theta_pred: 1.000000, 0.000000, -0.109330, 0.000000, 1.000000, 0.065008, 0.000

9060
9070
9080
9090
9100
theta_pred: 1.000000, 0.000000, 0.101635, 0.000000, 1.000000, 0.049268, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 5.000000, 0.000000, 0.000000
Computed images
9110
9120
9130
9140
9150
9160
9170
9180
9190
9200
theta_pred: 1.000000, 0.000000, 0.087199, 0.000000, 1.000000, -0.010701, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -1.000000, 0.000000, 1.000000, -14.000000, 0.000000, 0.000000
Computed images
9210
9220
9230
9240
9250
9260
9270
9280
9290
9300
theta_pred: 1.000000, 0.000000, 0.155717, 0.000000, 1.000000, -0.000102, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -2.000000, 0.000000, 1.000000, 5.000000, 0.000000, 0.000000
Computed images
9310
9320
9330
9340
9350
9360
9370
9380
9390
9400
theta_pred: 1.000000, 0.000000, 0.126805, 0.000000, 1.000000, -0.037513, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 6.000000, 0.000000, 1.000000, 6.000000, 0.000000, 0.000000
Computed images
9410
9420
9430
9440
9450
9

12010
12020
12030
12040
12050
12060
12070
12080
12090
12100
theta_pred: 1.000000, 0.000000, -0.044480, 0.000000, 1.000000, -0.102405, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -7.000000, 0.000000, 1.000000, 11.000000, 0.000000, 0.000000
Computed images
12110
12120
12130
12140
12150
12160
12170
12180
12190
12200
theta_pred: 1.000000, 0.000000, 0.034001, 0.000000, 1.000000, -0.137365, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -3.000000, 0.000000, 1.000000, -11.000000, 0.000000, 0.000000
Computed images
12210
12220
12230
12240
12250
12260
12270
12280
12290
12300
theta_pred: 1.000000, 0.000000, 0.069997, 0.000000, 1.000000, -0.098709, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -13.000000, 0.000000, 1.000000, -15.000000, 0.000000, 0.000000
Computed images
12310
12320
12330
12340
12350
12360
12370
12380
12390
12400
theta_pred: 1.000000, 0.000000, 0.092114, 0.000000, 1.000000, -0.327282, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -7.000000, 0.000000, 1.000000

...

Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
18010
18020
18030
18040
18050
18060
18070
18080
18090
18100
theta_pred: 1.000000, 0.000000, 0.109527, 0.000000, 1.000000, -0.091494, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 6.000000, 0.000000, 1.000000, -12.000000, 0.000000, 0.000000
Computed images
18110
18120
18130
18140
18150
18160
18170
18180
18190
18200
theta_pred: 1.000000, 0.000000, 0.118025, 0.000000, 1.000000, -0.092078, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -4.000000, 0.000000, 1.000000, 0.000000, 0.000000, 0.000000
Computed images
18210
18220
18230
18240
18250
18260
18270
18280
18290
18300
theta_pred: 1.000000, 0.000000, 0.137760, 0.000000, 1.000000, -0.051189, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 10.000000, 0.000000, 1.000000, 15.000000, 0.000000, 0.000000
Computed images
18310
18320
18330
18340
18350
18360
18370
18380
18390
18400
theta_pred: 1.000000, 0.000000, 0.15008

Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
21010
21020
21030
21040
21050
21060
21070
21080
21090
21100
theta_pred: 1.000000, 0.000000, 0.099868, 0.000000, 1.000000, -0.097688, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 2.000000, 0.000000, 1.000000, 15.000000, 0.000000, 0.000000
Computed images
21110
21120
21130
21140
21150
21160
21170
21180
21190
21200
theta_pred: 1.000000, 0.000000, 0.067090, 0.000000, 1.000000, -0.067945, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, -16.000000, 0.000000, 1.000000, 6.000000, 0.000000, 0.000000
Computed images
21210
21220
21230
21240
21250
21260
21270
21280
21290
21300
theta_pred: 1.000000, 0.000000, 0.089081, 0.000000, 1.000000, -0.052818, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 10.000000, 0.000000, 1.000000, -4.000000, 0.000000, 0.000000
Computed images
21310
21320
21330
21340
21350
21360
21370
21380
21390
21400
theta_pred: 1.000000, 0.000000, 0.14548

Validation pixel error 0.192665
Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
24010
24020
24030
24040
24050
24060
24070
24080
24090
24100
theta_pred: 1.000000, 0.000000, 0.047796, 0.000000, 1.000000, 0.074925, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 11.000000, 0.000000, 1.000000, -10.000000, 0.000000, 0.000000
Computed images
24110
24120
24130
24140
24150
24160
24170
24180
24190
24200
theta_pred: 1.000000, 0.000000, 0.108984, 0.000000, 1.000000, 0.044918, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 7.000000, 0.000000, 1.000000, -9.000000, 0.000000, 0.000000
Computed images
24210
24220
24230
24240
24250
24260
24270
24280
24290
24300
theta_pred: 1.000000, 0.000000, 0.179542, 0.000000, 1.000000, 0.097619, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 6.000000, 0.000000, 1.000000, 16.000000, 0.000000, 0.000000
Computed images
24310
24320
24330
24340
24350
24360
24370
24380
24390
24400
theta_pred

Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
27010
27020
27030
27040
27050
27060
27070
27080
27090
27100
theta_pred: 1.000000, 0.000000, 0.007266, 0.000000, 1.000000, 0.153424, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 7.000000, 0.000000, 1.000000, -11.000000, 0.000000, 0.000000
Computed images
27110
27120
27130
27140
27150
27160
27170
27180
27190
27200
theta_pred: 1.000000, 0.000000, 0.072788, 0.000000, 1.000000, 0.081498, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 3.000000, 0.000000, 1.000000, -7.000000, 0.000000, 0.000000
Computed images
27210
27220
27230
27240
27250
27260
27270
27280
27290
27300
theta_pred: 1.000000, 0.000000, 0.023817, 0.000000, 1.000000, 0.024897, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 8.000000, 0.000000, 1.000000, -9.000000, 0.000000, 0.000000
Computed images
27310
27320
27330
27340
27350
27360
27370
27380
27390
27400
theta_pred: 1.000000, 0.000000, -0.092568, 

Validation pixel error 0.192200
Computed histograms
Model saved in ./../../../datasets/isbi//realignment/train_transform/translation/run-21/model.ckpt
30010
30020
30030
30040
30050
30060
30070
30080
30090
30100
theta_pred: 1.000000, 0.000000, -0.005766, 0.000000, 1.000000, 0.078332, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 9.000000, 0.000000, 1.000000, -15.000000, 0.000000, 0.000000
Computed images
30110
30120
30130
30140
30150
30160
30170
30180
30190
30200
theta_pred: 1.000000, 0.000000, -0.003544, 0.000000, 1.000000, 0.033060, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 7.000000, 0.000000, 1.000000, -14.000000, 0.000000, 0.000000
Computed images
30210
30220
30230
30240
30250
30260
30270
30280
30290
30300
theta_pred: 1.000000, 0.000000, 0.041087, 0.000000, 1.000000, 0.041095, 0.000000, 0.000000
theta_true: 1.000000, 0.000000, 8.000000, 0.000000, 1.000000, 6.000000, 0.000000, 0.000000
Computed images
30310
30320
30330
30340
30350
30360
30370
30380
30390
30400
theta_pre